In [0]:
%run "/Shared/WeatherWatts/config/kafka keys"

In [0]:
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)


def acked(err, msg):
    """ 
        Error callback is used for generic issues for producer errors. 
        
        Parameters:
            err (err): Error flag.
            msg (str): Error message that was part of the callback.
    """
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))

In [0]:
s1 = 'new_weather_group2'

In [0]:
#Kakfa Class Setup.
c = Consumer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': s1,
    'default.topic.config': {'auto.offset.reset': 'latest'},
    'error_cb': error_cb,
})

topic_part = TopicPartition(confluentTopicName, 0)
c.assign([topic_part])
c.subscribe([confluentTopicName])

In [0]:
a_dict = {}

kafkaListDictionaries = []
# Once we have a list of dictionaries, we can use this to create a DataFrame instance

while(True):
    try:
        msg = c.poll(timeout=10)
        if msg is None:
            break
        elif msg.error():
            print("Consumer error: {}".format(msg.error()))
            break
        else:
            a_dict = json.loads('{}'.format(msg.value().decode('utf-8')))
            kafkaListDictionaries.append(a_dict)
    except Exception as e:
        print(e)

for message in kafkaListDictionaries:
    print(message)

c.close()

{'temp': 75.6, 'feelslike': 75.6, 'humidity': 60.99, 'dew': 61.2, 'precip': 0.0, 'precipprob': 10.0, 'snow': 0.0, 'snowdepth': 0.0, 'windspeed': 4.5, 'winddir': 60.0, 'sealevelevelpressure': 1016.0, 'visibility': 10.1, 'cloudcover': 25.3, 'solarradiation': 103.0, 'uvindex': 1.0, 'year': 2023, 'month': 5.0, 'days': 0.0, 'hour': 1.0, 'quarter': 2.0, 'dayofyear': 142, 'Adj_Close_3D_lag': 79.02999877929688, 'Adj_Close_30D_lag': 86.27999877929688, 'Adj_Close_60D_lag': 79.55000305175781, 'Holiday_ChristmasDay': 0.0, 'Holiday_ChristmasDaysubstitute': 0.0, 'Holiday_ChristmasEve': 0.0, 'Holiday_CincodeMayo': 0.0, 'Holiday_ColumbusDay': 0.0, 'Holiday_DayAfterChristmasDay': 0.0, 'Holiday_EasterMonday': 0.0, 'Holiday_EasterSunday': 0.0, 'Holiday_ElectionDay': 0.0, 'Holiday_ElectionDayGeneralElection': 0.0, 'Holiday_FathersDay': 0.0, 'Holiday_FlagDay': 0.0, 'Holiday_Halloween': 0.0, 'Holiday_InaugurationDay': 0.0, 'Holiday_IndependenceDay': 0.0, 'Holiday_IndependenceDaysubstitute': 0.0, 'Holiday_Ju

In [0]:
len(kafkaListDictionaries)

Out[45]: 96

In [0]:
df_from_list_of_dictionaries = spark.createDataFrame(kafkaListDictionaries)
df_from_list_of_dictionaries = df_from_list_of_dictionaries.limit(272)


display(df_from_list_of_dictionaries)

Adj_Close_30D_lag,Adj_Close_3D_lag,Adj_Close_60D_lag,Date,Holiday_ChristmasDay,Holiday_ChristmasDaysubstitute,Holiday_ChristmasEve,Holiday_CincodeMayo,Holiday_ColumbusDay,Holiday_DayAfterChristmasDay,Holiday_EasterMonday,Holiday_EasterSunday,Holiday_ElectionDay,Holiday_ElectionDayGeneralElection,Holiday_FathersDay,Holiday_FlagDay,Holiday_Halloween,Holiday_InaugurationDay,Holiday_IndependenceDay,Holiday_IndependenceDaysubstitute,Holiday_Juneteenth,Holiday_Juneteenthsubstitute,Holiday_LaborDay,Holiday_MartinLutherKingJrDay,Holiday_MemorialDay,Holiday_MothersDay,Holiday_NewYearsDay,Holiday_NewYearsDaysubstitute,Holiday_NewYearsEve,Holiday_PresidentsDay,Holiday_StPatricksDay,Holiday_SuperTuesday,Holiday_TaxDay,Holiday_ThanksgivingDay,Holiday_ValentinesDay,Holiday_VeteransDay,Holiday_VeteransDaysubstitute,IsWeekend_True,State,Temp_24_hours_ahead,cloudcover,dayofyear,days,demand_3d_lag,dew,feelslike,hour,humidity,month,precip,precipprob,quarter,sealevelevelpressure,snow,snowdepth,solarradiation,temp,uvindex,visibility,winddir,windspeed,year
86.27999877929688,79.02999877929688,79.55000305175781,2023-05-22T01:00:00.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TX,80.0,25.3,142,0.0,58282.0,61.2,75.6,1.0,60.99,5.0,0.0,10.0,2.0,1016.0,0.0,0.0,103.0,75.6,1.0,10.1,60.0,4.5,2023
86.27999877929688,79.02999877929688,79.55000305175781,2023-05-22T02:00:00.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TX,77.4,20.7,142,0.0,56281.0,62.0,73.5,2.0,67.25,5.0,0.0,10.0,2.0,1016.0,0.0,0.0,64.0,73.5,0.0,10.1,70.0,3.6,2023
86.27999877929688,79.02999877929688,79.55000305175781,2023-05-22T03:00:00.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TX,74.7,16.0,142,0.0,54137.0,62.5,71.3,3.0,73.73,5.0,0.0,10.0,2.0,1016.0,0.0,0.0,26.0,71.3,0.0,10.1,80.0,2.7,2023
86.27999877929688,79.02999877929688,79.55000305175781,2023-05-22T04:00:00.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TX,73.3,15.3,142,0.0,50341.0,62.3,69.7,4.0,77.42,5.0,0.0,10.0,2.0,1017.0,0.0,0.0,22.0,69.7,0.0,10.1,60.0,2.7,2023
86.27999877929688,79.02999877929688,79.55000305175781,2023-05-22T05:00:00.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TX,71.7,14.7,142,0.0,46216.0,62.0,68.3,5.0,80.3,5.0,0.0,10.0,2.0,1017.0,0.0,0.0,17.0,68.3,0.0,10.1,40.0,2.7,2023
86.27999877929688,79.02999877929688,79.55000305175781,2023-05-22T06:00:00.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TX,70.2,14.0,142,0.0,42909.0,61.8,66.6,6.0,84.38,5.0,0.0,10.0,2.0,1017.0,0.0,0.0,13.0,66.6,0.0,10.1,20.0,2.7,2023
86.27999877929688,79.02999877929688,79.55000305175781,2023-05-22T07:00:00.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TX,69.3,14.3,142,0.0,40698.0,61.4,65.9,7.0,85.42,5.0,0.0,4.0,2.0,1016.0,0.0,0.0,9.0,65.9,0.0,10.1,20.0,2.7,2023
86.27999877929688,79.02999877929688,79.55000305175781,2023-05-22T08:00:00.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TX,68.6,14.7,142,0.0,39198.0,61.1,65.0,8.0,87.01,5.0,0.0,4.0,2.0,1016.0,0.0,0.0,4.0,65.0,0.0,10.1,20.0,2.7,2023
86.27999877929688,79.02999877929688,79.55000305175781,2023-05-22T09:00:00.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TX,67.7,15.0,142,0.0,38252.0,60.7,64.3,9.0,88.1,5.0,0.0,4.0,2.0,1016.0,0.0,0.0,0.0,64.3,0.0,10.1,20.0,2.7,

In [0]:
%run "/Shared/WeatherWatts/config/SQL Keys"

In [0]:
df_from_list_of_dictionaries = df_from_list_of_dictionaries.toPandas()

In [0]:
df_ny = df_from_list_of_dictionaries.query('State == "NY"')
df_tx = df_from_list_of_dictionaries.query('State == "TX"')
df_ca = df_from_list_of_dictionaries.query('State == "CA"')
df_fl = df_from_list_of_dictionaries.query('State == "FL"')

In [0]:
df_ca

,Adj_Close_30D_lag,Adj_Close_3D_lag,Adj_Close_60D_lag,Date,Holiday_ChristmasDay,Holiday_ChristmasDaysubstitute,Holiday_ChristmasEve,Holiday_CincodeMayo,Holiday_ColumbusDay,Holiday_DayAfterChristmasDay,...,sealevelevelpressure,snow,snowdepth,solarradiation,temp,uvindex,visibility,winddir,windspeed,year


In [0]:
display(df_ny)

Adj_Close_30D_lag,Adj_Close_3D_lag,Adj_Close_60D_lag,Date,Holiday_ChristmasDay,Holiday_ChristmasDaysubstitute,Holiday_ChristmasEve,Holiday_CincodeMayo,Holiday_ColumbusDay,Holiday_DayAfterChristmasDay,Holiday_EasterMonday,Holiday_EasterSunday,Holiday_ElectionDay,Holiday_ElectionDayGeneralElection,Holiday_FathersDay,Holiday_FlagDay,Holiday_Halloween,Holiday_InaugurationDay,Holiday_IndependenceDay,Holiday_IndependenceDaysubstitute,Holiday_Juneteenth,Holiday_Juneteenthsubstitute,Holiday_LaborDay,Holiday_MartinLutherKingJrDay,Holiday_MemorialDay,Holiday_MothersDay,Holiday_NewYearsDay,Holiday_NewYearsDaysubstitute,Holiday_NewYearsEve,Holiday_PresidentsDay,Holiday_StPatricksDay,Holiday_SuperTuesday,Holiday_TaxDay,Holiday_ThanksgivingDay,Holiday_ValentinesDay,Holiday_VeteransDay,Holiday_VeteransDaysubstitute,IsWeekend_True,State,Temp_24_hours_ahead,cloudcover,dayofyear,days,demand_3d_lag,dew,feelslike,hour,humidity,month,precip,precipprob,quarter,sealevelevelpressure,snow,snowdepth,solarradiation,temp,uvindex,visibility,winddir,windspeed,year
85.4800033569336,78.45999908447266,77.58999633789062,2023-05-20T04:00:00.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NY,65.0,64.0,140,5.0,14127.0,52.1,60.0,4.0,75.07,5.0,0.0,10.0,2.0,1024.0,0.0,0.0,0.0,60.0,0.0,10.1,150.0,8.1,2023
85.4800033569336,78.45999908447266,77.58999633789062,2023-05-20T05:00:00.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NY,63.9,67.0,140,5.0,13341.0,53.0,58.9,5.0,80.64,5.0,0.0,10.0,2.0,1024.0,0.0,0.0,0.0,58.9,0.0,10.1,140.0,6.9,2023
85.4800033569336,78.45999908447266,77.58999633789062,2023-05-20T06:00:00.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NY,63.1,71.0,140,5.0,12825.0,53.0,60.0,6.0,77.6,5.0,0.0,24.0,2.0,1022.0,0.0,0.0,0.0,60.0,0.0,10.1,130.0,6.9,2023
85.4800033569336,78.45999908447266,77.58999633789062,2023-05-20T07:00:00.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NY,61.6,76.0,140,5.0,12518.0,54.1,58.9,7.0,83.9,5.0,0.0,24.0,2.0,1022.0,0.0,0.0,0.0,58.9,0.0,6.9,120.0,6.9,2023
85.4800033569336,78.45999908447266,77.58999633789062,2023-05-20T08:00:00.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NY,60.3,81.0,140,5.0,12393.0,54.1,58.9,8.0,83.9,5.0,0.0,24.0,2.0,1021.0,0.0,0.0,0.0,58.9,0.0,3.9,110.0,5.8,2023
85.4800033569336,78.45999908447266,77.58999633789062,2023-05-20T09:00:00.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NY,58.9,86.0,140,5.0,12535.0,54.1,58.9,9.0,83.9,5.0,0.0,40.0,2.0,1021.0,0.0,0.0,0.0,58.9,0.0,3.9,100.0,5.8,2023
85.4800033569336,78.45999908447266,77.58999633789062,2023-05-20T10:00:00.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NY,59.6,90.0,140,5.0,13186.0,54.9,60.0,10.0,83.42,5.0,0.0,40.0,2.0,1020.0,0.0,0.0,7.0,60.0,0.0,1.6,100.0,5.8,2023
85.4800033569336,78.45999908447266,77.58999633789062,2023-05-20T11:00:00.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NY,60.3,93.0,140,5.0,14278.0,56.0,60.0,11.0,86.75,5.0,0.012,40.0,2.0,1020.0,0.0,0.0,22.0,60.0,0.0,2.0,90.0,6.9,2023
85.4800033569336,78.45999908447266,77.58999633789062,2023-05-20T12:00:00.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NY,61.1,97.0,140,5.0,15094.0,56.9,61.1,12.0,86.25,5.0,0.012,78.0,2.0,1020.0,0.0,0.0,37.0,61.1,0.0,1.6,90.0,6.9,2023


In [0]:
df_ny = spark.createDataFrame(df_ny)
df_tx = spark.createDataFrame(df_tx)
df_ca = spark.createDataFrame(df_ca)
df_fl = spark.createDataFrame(df_fl)

In [0]:
df_ny.write.format("jdbc").option(
    "url", f"jdbc:sqlserver://{server}:1433;databaseName={database};"
    ) \
    .mode("append") \
    .option("dbtable", tableNY) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-4465790401556832>:1
----> 1 df_ny.write.format("jdbc").option(
      2     "url", f"jdbc:sqlserver://{server}:1433;databaseName={database};"
      3     ) \
      4     .mode("append") \
      5     .option("dbtable", tableNY) \
      6     .option("user", user) \
      7     .option("password", password) \
      8     .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
      9     .save()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwrite

In [0]:
df_ca.write.format("jdbc").option(
    "url", f"jdbc:sqlserver://{server}:1433;databaseName={database};"
    ) \
    .mode("append") \
    .option("dbtable", tableCA) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-340839472825198>:1
----> 1 df_ca.write.format("jdbc").option(
      2     "url", f"jdbc:sqlserver://{server}:1433;databaseName={database};"
      3     ) \
      4     .mode("append") \
      5     .option("dbtable", tableCA) \
      6     .option("user", user) \
      7     .option("password", password) \
      8     .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
      9     .save()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter

In [0]:
df_tx.write.format("jdbc").option(
    "url", f"jdbc:sqlserver://{server}:1433;databaseName={database};"
    ) \
    .mode("append") \
    .option("dbtable", tableTX) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-340839472825200>:1
----> 1 df_tx.write.format("jdbc").option(
      2     "url", f"jdbc:sqlserver://{server}:1433;databaseName={database};"
      3     ) \
      4     .mode("append") \
      5     .option("dbtable", tableTX) \
      6     .option("user", user) \
      7     .option("password", password) \
      8     .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
      9     .save()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter

In [0]:
df_fl.write.format("jdbc").option(
    "url", f"jdbc:sqlserver://{server}:1433;databaseName={database};"
    ) \
    .mode("append") \
    .option("dbtable", tableFL) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-340839472825201>:1
----> 1 df_fl.write.format("jdbc").option(
      2     "url", f"jdbc:sqlserver://{server}:1433;databaseName={database};"
      3     ) \
      4     .mode("append") \
      5     .option("dbtable", tableFL) \
      6     .option("user", user) \
      7     .option("password", password) \
      8     .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
      9     .save()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter